In [ ]:
import pandas as pd # data processing
import numpy as np

#IMDB Movie Reviews dataset.
imdb_dataset = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv',  encoding='utf-8')

########################################################################################################################################

#Twitter US Airline dataset.
USAirline_dataset = pd.read_csv('/kaggle/input/twitter-airline-sentiment/Tweets.csv',  encoding='utf-8')
# combaine negative reason with  tweet
USAirline_dataset['review'] = USAirline_dataset['negativereason'].fillna('') + ' ' + USAirline_dataset['text'] 
USAirline_dataset['sentiment'] = USAirline_dataset['airline_sentiment']
# Selecting specific columns from the original dataset
USAirline_dataset = USAirline_dataset[['review', 'sentiment']]

########################################################################################################################################

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from termcolor import cprint
import seaborn as sns
import warnings   

warnings.filterwarnings('ignore')
%matplotlib inline

# Display the count of different sentiments
cprint("Total number of sentiments of tweets:", 'green')
print(imdb_dataset.sentiment.value_counts())

# Set up the figure for subplots
plt.figure(figsize=(14, 5))  # Adjust overall figure size

# First subplot: Countplot of sentiments
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first plot
ax1 = sns.countplot(x='sentiment', data=imdb_dataset)
# Iterate through the list of axes' patches (bars)
for p in ax1.patches:
    ax1.annotate(format(p.get_height(), '.0f'),  # Get the height of each bar (count)
                 (p.get_x() + p.get_width() / 2., p.get_height()),  # Position for the label
                 ha = 'center', va = 'center',  # Center alignment
                 xytext = (0, 9),  # Distance from the top of the bar
                 textcoords = 'offset points')

# Second subplot: Pie chart of sentiments distribution
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second plot
colors = sns.color_palette('husl', 10)  # Define a color palette
sentiment_counts = imdb_dataset['sentiment'].value_counts()
sentiment_counts.plot(
    kind='pie',
    colors=colors,
    labels=sentiment_counts.index,  # Automatically use labels from the sentiment column
    explode=[0.05] * len(sentiment_counts),  # Dynamically generate explode values based on the number of categories
    shadow=True,
    autopct='%.2f%%',  # Show the percentage of each sentiment
    fontsize=12,
    figsize=(12, 5)
)

plt.tight_layout()  # Adjust subplots to fit into the figure area.
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from termcolor import cprint

# Display the count of different sentiments
cprint("Total number of sentiments of tweets:", 'green')
print(USAirline_dataset.sentiment.value_counts())

# Set up the figure for subplots
plt.figure(figsize=(14, 5))  # Adjust overall figure size

# First subplot: Countplot of sentiments
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first plot
ax1 = sns.countplot(x='sentiment', data=USAirline_dataset)
# Iterate over each bar in the countplot and annotate the height (count)
for bar in ax1.patches:
    ax1.annotate(f'{int(bar.get_height())}',  # The count as text
                 (bar.get_x() + bar.get_width() / 2, bar.get_height()),  # Position for the annotation
                 ha='center', va='center',  # Center the text horizontally and vertically
                 xytext=(0, 8),  # Position text slightly above the bar
                 textcoords='offset points')

# Second subplot: Pie chart of sentiments distribution
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second plot
colors = sns.color_palette('husl', 10)  # Define a color palette
sentiment_counts = USAirline_dataset['sentiment'].value_counts()
sentiment_counts.plot(
    kind='pie',
    colors=colors,
    labels=sentiment_counts.index,  # Automatically use labels from the sentiment column
    explode=[0.05] * len(sentiment_counts),  # Dynamically generate explode values based on the number of categories
    shadow=True,
    autopct='%.2f%%',  # Show the percentage of each sentiment
    fontsize=12,
    figsize=(12, 5)
)

plt.tight_layout()  # Adjust subplots to fit into the figure area.
plt.show()

# Creating Data cleaning protocol

In [ ]:
import os
import numpy as np
import pandas as pd
import emoji
from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
import warnings
# Ensure this is imported for handling punctuation
import re,string,unicodedata

import spacy
from spacy.lang.en import English

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import seaborn as sns
import matplotlib.pyplot as plt

# Your contractions dictionary
contractions_dict = {
    "I'm": "I am",
    "I'm'a": "I am about to",
    "I'm'o": "I am going to",
    "I've": "I have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'd": "I would",
    "I'd've": "I would have",
    "Whatcha": "What are you",
    "amn't": "am not",
    "ain't": "are not",
    "aren't": "are not",
    "'cause": "because",
    "can't": "cannot",
    "can't've": "cannot have",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "daren't": "dare not",
    "daresn't": "dare not",
    "dasn't": "dare not",
    "didn't": "did not",
    "didn’t": "did not",
    "don't": "do not",
    "don’t": "do not",
    "doesn't": "does not",
    "e'er": "ever",
    "everyone's": "everyone is",
    "finna": "fixing to",
    "gimme": "give me",
    "gon't": "go not",
    "gonna": "going to",
    "gotta": "got to",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he've": "he have",
    "he's": "he is",
    "he'll": "he will",
    "he'll've": "he will have",
    "he'd": "he would",
    "he'd've": "he would have",
    "here's": "here is",
    "how're": "how are",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how's": "how is",
    "how'll": "how will",
    "isn't": "is not",
    "it's": "it is",
    "'tis": "it is",
    "'twas": "it was",
    "it'll": "it will",
    "it'll've": "it will have",
    "it'd": "it would",
    "it'd've": "it would have",
    "kinda": "kind of",
    "let's": "let us",
    "luv": "love",
    "ma'am": "madam",
    "may've": "may have",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "ne'er": "never",
    "o'": "of",
    "o'clock": "of the clock",
    "ol'": "old",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "o'er": "over",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shalln't": "shall not",
    "shan't've": "shall not have",
    "she's": "she is",
    "she'll": "she will",
    "she'd": "she would",
    "she'd've": "she would have",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "somebody's": "somebody is",
    "someone's": "someone is",
    "something's": "something is",
    "sux": "sucks",
    "that're": "that are",
    "that's": "that is",
    "that'll": "that will",
    "that'd": "that would",
    "that'd've": "that would have",
    "'em": "them",
    "there're": "there are",
    "there's": "there is",
    "there'll": "there will",
    "there'd": "there would",
    "there'd've": "there would have",
    "these're": "these are",
    "they're": "they are",
    "they've": "they have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they'd": "they would",
    "they'd've": "they would have",
    "this's": "this is",
    "this'll": "this will",
    "this'd": "this would",
    "those're": "those are",
    "to've": "to have",
    "wanna": "want to",
    "wasn't": "was not",
    "we're": "we are",
    "we've": "we have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we'd": "we would",
    "we'd've": "we would have",
    "weren't": "were not",
    "what're": "what are",
    "what'd": "what did",
    "what've": "what have",
    "what's": "what is",
    "what'll": "what will",
    "what'll've": "what will have",
    "when've": "when have",
    "when's": "when is",
    "where're": "where are",
    "where'd": "where did",
    "where've": "where have",
    "where's": "where is",
    "which's": "which is",
    "who're": "who are",
    "who've": "who have",
    "who's": "who is",
    "who'll": "who will",
    "who'll've": "who will have",
    "who'd": "who would",
    "who'd've": "who would have",
    "why're": "why are",
    "why'd": "why did",
    "why've": "why have",
    "why's": "why is",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "you're": "you are",
    "you've": "you have",
    "you'll've": "you shall have",
    "you'll": "you will",
    "you'd": "you would",
    "you'd've": "you would have",
    # chat_words
    # IMDb-specific chat words
    "CGI": "Computer-Generated Imagery",
    "VFX": "Visual Effects",
    "OST": "Original Soundtrack",
    "GOAT": "Greatest Of All Time",
    # US Airline/Tweets-specific chat words
    "ETA": "Estimated Time of Arrival",
    "ETD": "Estimated Time of Departure",
    "Layover": "A break between flights on a longer journey",
    "FFP": "Frequent Flyer Program",
    "TSA": "Transportation Security Administration",
    # Sentiment140-specific chat words
    "LOL": "Laugh out loud",
    "OMG": "Oh my God/goodness",
    "SMH": "Shaking my head",
    "LMAO": "Laughing my ass off",
    "TTYL": "Talk to you later",
    "BRB": "Be right back",
    "IMO/IMHO": "In my (humble) opinion",
    "NSFW": "Not safe for work",
    ###
    "BRB": "Be right back",
    "BTW": "By the way",
    "TTYL": "Talk to you later",
    "OMW": "On my way",
    "SMH/SMDH": "Shaking my head/shaking my darn head",
    "LOL": "Laugh out loud",
    "TBD": "To be determined", 
    "IMHO/IMO": "In my humble opinion",
    "HMU": "Hit me up",
    "IIRC": "If I remember correctly",
    "LMK": "Let me know", 
    "OG": "Original gangsters (used for old friends)",
    "FTW": "For the win", 
    "NVM": "Nevermind",
    "OOTD": "Outfit of the day", 
    "Ngl": "Not gonna lie",
    "Rq": "real quick", 
    "Iykyk": "If you know, you know",
    "Ong": "On god (I swear)", 
    "YAAAS": "Yes!", 
    "Brt": "Be right there",
    "Sm": "So much",
    "Ig": "I guess",
    "Wya": "Where you at",
    "Istg": "I swear to god",
    "Hbu": "How about you",
    "Atm": "At the moment",
    "Asap": "As soon as possible",
    "Fyi": "For your information"

 }

# 'contractions_dict' is defined
contractions_pattern = re.compile('(%s)' % '|'.join([re.escape(k) for k in contractions_dict.keys()]), flags=re.IGNORECASE)


def Data_cleaning_protocol(text):
    def remove_html(text):
        # First, remove HTML to avoid HTML tags affecting other text processing steps.
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', MarkupResemblesLocatorWarning)
            return BeautifulSoup(text, "html.parser").get_text()

    def expand_contractions(text):
        # Expand contractions early to ensure that any punctuation involved in contractions is handled correctly.
        def replace(match):
            return contractions_dict.get(match.group(0).lower(), match.group(0))
        return contractions_pattern.sub(replace, text)
    
    def handle_emojis(text):
        # Handle emojis early to convert them into text, which can be processed by subsequent steps.
        text = emoji.demojize(text, delimiters=(" ", " "))
        return text

    def remove_urls_usernames(text):
        # Remove URLs and usernames to prevent them from being altered by later steps like removing special characters.
        return re.sub(r'https?://\S+|www\.\S+|@[^\s]+', '', text)

    def remove_html_linebreaks(text):
        # Handle HTML line breaks after removing HTML content but before altering the text further.
        # This targets a specific HTML line break pattern for conversion to space.
        linebreaks = "<br /><br />"
        return re.sub(linebreaks, " ", text)

    def to_lowercase(text):
        # Convert to lowercase early to standardize the case for all subsequent operations.
        return text.lower()
    
    def remove_square_brackets(text):
        # Remove text within square brackets before general special character removal to target specific unwanted text.
        return re.sub(r'\[[^]]*\]', '', text)

    def remove_special_characters(text, remove_digits=True):
        # Removing special characters (and optionally digits) after specific patterns like URLs, usernames,
        # and square brackets have been addressed.
        pattern = r'[^a-zA-Z\s]' if remove_digits else r'[^a-zA-Z0-9\s]'
        return re.sub(pattern, '', text)

    def reduce_character_sequences(text):
        # Reduce sequences of repetitive characters after case normalization and special character removal
        # to clean up any resulting awkwardness from previous steps.
        sequencePattern = r"(.)\1\1+"
        seqReplacePattern = r"\1\1"
        return re.sub(sequencePattern, seqReplacePattern, text)

    def remove_hash_symbol(text):
        # Remove the '#' symbol from hashtags, allowing for clean word extraction, performed after other text cleanups.
        return re.sub(r'#(\S+)', r'\1', text)

    # Applying all preprocessing steps in logical order
    text = remove_html(text)
    text = handle_emojis(text)
    text = expand_contractions(text)
    text = remove_urls_usernames(text)
    text = remove_html_linebreaks(text)
    text = to_lowercase(text)
    text = remove_square_brackets(text)
    text = remove_special_characters(text)
    # Redundant removal of words containing numbers after special characters have been handled
    text = re.sub('\w*\d\w*', '', text)
    # Newline characters are removed here to clean up any formatting leftovers.
    text = re.sub('\n', '', text)
    text = reduce_character_sequences(text)
    text = remove_hash_symbol(text)

    return text

#  Stopwords Remove (Customized)

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

stopword_set = set(nltk.corpus.stopwords.words('english'))  # English stopwords set for efficiency
print('nltk stopwords words \n', stopword_set)
print(" ")

#  Keeping some necessary words
words_to_keep = {'don', 'wouldn', "wasn't", 'mustn', 'against', "mightn't", "aren't", 'won', 'aren', "didn't", 'should', 'haven', 
                 "don't", 'off', "won't", 'hadn', 'didn', 'shan', 'below', 'doesn', 'most', "shan't", 'than', 'mightn', "isn't",
                 'needn', 'nor',  "wouldn't", 'not', 'above', "needn't", 'under', 'no', 'couldn', 'down', "couldn't", "should've",
                 "mustn't", 'wasn', 'shouldn', 'but', 'ain', "hadn't", 'out', "weren't", "doesn't", 'over', 'hasn','weren', 'same', 
                 "shouldn't", "haven't", 'isn', "hasn't"}

# Remove the selected words from the stopword set
stopword_set -= words_to_keep
print('After keeping some necessary words nltk stopwords words \n', stopword_set)

# Define the remove_stopwords function
def custom_remove_stopwords(text, stopword_set):
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stopword_set]
    return ' '.join(filtered_tokens)

# Lemmatize

In [ ]:
import spacy
import pandas as pd

# Load the spaCy model
# nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Define the lemmatize_texts function
def lemmatize_texts(texts, nlp):
    docs = nlp.pipe(texts)    #     docs = nlp(texts)
    return [' '.join([token.lemma_ for token in doc]) for doc in docs]

# Apply preprocessing  to IMDb dataset

In [ ]:
# Apply to IMDb dataset

# Apply Data_cleaning_protocol
imdb_dataset['review_P'] = imdb_dataset['review'].apply(Data_cleaning_protocol)
# Apply Data_cleaning_protocol + custom_remove_stopwords
imdb_dataset['review_PS'] = imdb_dataset['review_P'].apply(lambda x: custom_remove_stopwords(x, stopword_set))
# Apply Data_cleaning_protocol + custom_remove_stopwords + lemmatization
imdb_dataset['review_PSL'] = lemmatize_texts(imdb_dataset['review_PS'], nlp)
# Apply Data_cleaning_protocol +lemmatization
imdb_dataset['review_PL'] = lemmatize_texts(imdb_dataset['review_P'], nlp)

## Save processed Data if you want
imdb_dataset.to_csv("imdb_dataset_processed.csv", index=False)
# imdb_dataset.to_csv("PS(remove_stopwords)_imdb_dataset.csv", index=False)
# imdb_dataset.to_csv("PSL(lemmatize_texts_imdb)_dataset.csv", index=False)
print('------------->imdb_dataset: I AM DONE<-------------------')

# Apply preprocessing to USAirline dataset

In [ ]:
# Apply to USAirline dataset

# Apply Data_cleaning_protocol
USAirline_dataset['review_P'] = USAirline_dataset['review'].apply(Data_cleaning_protocol)
# Apply Data_cleaning_protocol + custom_remove_stopwords
USAirline_dataset['review_PS'] = USAirline_dataset['review_P'].apply(lambda x: custom_remove_stopwords(x, stopword_set))
# Apply Data_cleaning_protocol + custom_remove_stopwords + lemmatization
USAirline_dataset['review_PSL'] = lemmatize_texts(USAirline_dataset['review_PS'], nlp)
# Apply Data_cleaning_protocol +lemmatization
USAirline_dataset['review_PL'] = lemmatize_texts(USAirline_dataset['review_P'], nlp)

## Save processed Data if you want
USAirline_dataset.to_csv("USAirline_dataset_processed.csv", index=False)
# USAirline_dataset.to_csv("PS(remove_stopwords)_USAirline_dataset.csv", index=False)
# USAirline_dataset.to_csv("PSL(lemmatize_texts_imdb)_USAirline_dataset.csv", index=False)
print('------------->USAirline_dataset: I AM DONE<-------------------')